# Pruning the Model


## Importing all neccessary libararies


In [78]:
import pandas as pd
import numpy as np
import time
import os,sys
import plotly.graph_objs as go
sys.path.append('../')
from transformers import AutoModel
from src.pruning_class import pruning
from src.model_evaluation import evaluation
from src.dimensionality_reduction_class import dimensionality_reduction
from sentence_transformers import SentenceTransformer
from plotly.subplots import make_subplots

# Creating the object of evaluation class

In [ ]:
eval_object=evaluation()

# Loading the training data

In [68]:
df=pd.read_csv("..//Data/processed_data.csv")


list_data = df['tokenized_docstring'].tolist()

# Loading the testing data

In [67]:
queries=pd.read_csv("..//Data/query.csv")

# Creating the list to store the values

In [151]:
time_embedding_generation=[]
emebdding_size_MB=[]
model_size_MB=[]
layers=[]
accuracy=[]

# Loading the model on which pruning to be done

In [38]:
base_model=SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 4 Layer Model

In [2]:
# layer to be kept in base model
layers=[3, 5, 8, 11]

#Creating the object of distillation class by passing layers and model_name
prun_object=pruning(layers,base_model)

# function of distillation class that makes the model with 4 layers and saved in local drive
dist_object.pruning_model()


/Users/msaqib/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Loading the saved model from local drive and generatting the embeddings

In [5]:
# Loading the saved model
model_4_layer=SentenceTransformer('..//models/pruned_model_with_4_layer')

# generating the embeddings for this model and saving in the local drive
get_model_embedding(model_4_layer,"pruned_model_with_4_layer")

starting_time : 1713343941.56
ending_time : 1713344512.35
Embedding_shape : (113884, 768)
total_time_taken for embedding generation:  570.79 seconds 


# getting the size of 4 layer model

In [17]:

size=get_model_size("..//models/pruned_model_with_4_layer")

print(f"Size of 4 layer model is {(size/(1024*1024)):.2f} MB")

size of 4 layer model is 202.27 MB


# Getting the size of embeddings of 4 layer model

In [22]:
size_of_embeddings=get_embedding_size("..//embeddings/embeddings_pruned_model_with_4_layer.npy")

print(f"Size of embeddings of 4 layer model is {(size_of_embeddings/(1024*1024)):.2f} MB")

Size of embeddings of 4 layer model is 333.64 MB


# loading the values in different list

In [152]:
time_embedding_generation.append(570.79)
emebdding_size_MB.append(333.64)
model_size_MB.append(202.27)
layers.append(4)

# Architecture of 4 Layer Model

In [41]:
# Print the model summary to view its architecture
print(model_4_layer)


#Number of layers is 4 and Embedding vector dimnesion is 768 

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-3): 4 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_fe

# Generating the response for the testing data

In [79]:
# order in which column of databse will be present
column_order = ['Query', 'Docstrings', 'Code','Match']

#creating the dataframe for model_1 which consists of for particular query all top code and
#match column shows whether it is correctly retrieved or not
model_4_layer_query_response=pd.DataFrame(columns=column_order)

model_4_layer=SentenceTransformer('..//models/pruned_model_with_4_layer')

emb_4_layer=np.load("..//embeddings/embeddings_pruned_model_with_4_layer.npy")

print(emb_4_layer.shape)

for i in range(len(queries)):
    #getting the particular query from queries 
    query=queries.loc[i].Questions
    
    # getting the table dataframe that consists of docstring and code 
    #that has top match with query using cosine similarity
    table=eval_object.get_top_code_and_docstring(query,emb_4_layer,model_4_layer,list_data,df)
    
    #adding the column query with all values as 
    table=add_column(table,query)
    
    #adding columns Match that defines whether the code for given query is correctly retrieved or not
    table=add_column(table,'Match')
    
    #reordering the table for better view
    table=reorder_columns(table)
    
    #iterating in the table
    for index,row in table.iterrows():
        
        #fetching the current code for query
        result = row['Code']
        
        #getting the response from claude api
        response=eval_object.check_response(query,row['Code'])
    
        #Checking the value present in the response generated by the claude
        if 'YES' in response.content:
            response='YES'
        else :
            response='NO'
            
        #Giving the value of response in match column for current row
        table.at[index,'Match']=response
    
    #merging the table dataframe and model_1_query_response in single dataframe
    model_4_layer_query_response= pd.concat([model_4_layer_query_response, table], ignore_index=True)

(113884, 768)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# 6 Layers Model

In [23]:
# layers to keep in base model
layers=[1,3,4,7,10,11]

# Creating the obvject of distillation class
prun_object=pruning(layers,teacher_model)

# function of distillation class that makes the model with 6 layer
prun_object.pruning_model()


# Loading the model from local drive and generating the embeddings

In [27]:
model_6_layer=SentenceTransformer('..//models/pruning_model_with_6_layer')
get_model_embedding(model_6_layer,"pruning_model_with_6_layer")

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_6_layer and are newly initialized: ['encoder.layer.4.output.LayerNorm.bias', 'encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.3.attention.attn.k.bias', 'encoder.layer.2.attention.attn.v.weight', 'encoder.layer.4.attention.attn.k.weight', 'encoder.layer.4.output.dense.bias', 'encoder.layer.4.attention.attn.q.weight', 'encoder.layer.3.attention.attn.v.weight', 'encoder.layer.4.attention.attn.v.bias', 'encoder.layer.3.intermediate.dense.weight', 'encoder.layer.2.attention.attn.k.weight', 'encoder.layer.5.attention.attn.o.weight', 'encoder.layer.4.attention.LayerNorm.bias', 'encoder.layer.5.attention.LayerNorm.bias', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.5.intermediate.dense.bias', 'encoder.layer.5.output.LayerNorm.bias', 'encoder.layer.5.a

starting_time : 1713345927.98
ending_time : 1713346259.92
Embedding_shape : (113884, 768)
total_time_taken for embedding generation:  331.94 seconds 


# Getting the size of model with 6 layer


In [28]:
size=func_object.get_model_size("..//models/pruning_model_with_6_layer")

print(f"Size of 6 layer model is {(size/(1024*1024)):.2f} MB")

Size of 6 layer model is 148.19 MB


# Getting the size of embeddings

In [29]:
size_of_embeddings=func_object.get_embedding_size("..//embeddings/embeddings_pruning_model_with_6_layer.npy")

print(f"Size of embeddings of 6 layer model is {(size_of_embeddings/(1024*1024)):.2f} MB")

Size of embeddings of 6 layer model is 333.64 MB


# Loading the values in different list

In [153]:
time_embedding_generation.append(331.94)
emebdding_size_MB.append(333.64)
model_size_MB.append(148.19)
layers.append(6)

# Architecture of model with 6 layer

In [42]:
# Print the model summary to view its architecture
print(model_6_layer)


# Number of layers is 6 and Embedding vector dimnesion is 768 

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_6_layer and are newly initialized: ['encoder.layer.4.output.LayerNorm.bias', 'encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.3.attention.attn.k.bias', 'encoder.layer.2.attention.attn.v.weight', 'encoder.layer.4.attention.attn.k.weight', 'encoder.layer.4.output.dense.bias', 'encoder.layer.4.attention.attn.q.weight', 'encoder.layer.3.attention.attn.v.weight', 'encoder.layer.4.attention.attn.v.bias', 'encoder.layer.3.intermediate.dense.weight', 'encoder.layer.2.attention.attn.k.weight', 'encoder.layer.5.attention.attn.o.weight', 'encoder.layer.4.attention.LayerNorm.bias', 'encoder.layer.5.attention.LayerNorm.bias', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.5.intermediate.dense.bias', 'encoder.layer.5.output.LayerNorm.bias', 'encoder.layer.5.a

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-5): 6 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_fe

# Generating the response for the testing data

In [80]:
# order in which column of databse will be present
column_order = ['Query', 'Docstrings', 'Code','Match']

#creating the dataframe for model_1 which consists of for particular query all top code and
#match column shows whether it is correctly retrieved or not
model_6_layer_query_response=pd.DataFrame(columns=column_order)

model_6_layer=SentenceTransformer('..//models/pruned_model_with_6_layer')

emb_6_layer=np.load("..//embeddings/embeddings_pruned_model_with_6_layer.npy")

print(emb_6_layer.shape)

for i in range(len(queries)):
    #getting the particular query from queries 
    query=queries.loc[i].Questions
    
    # getting the table dataframe that consists of docstring and code 
    #that has top match with query using cosine similarity
    table=eval_object.get_top_code_and_docstring(query,emb_6_layer,model_6_layer,list_data,df)
    
    #adding the column query with all values as 
    table=add_column(table,query)
    
    #adding columns Match that defines whether the code for given query is correctly retrieved or not
    table=add_column(table,'Match')
    
    #reordering the table for better view
    table=reorder_columns(table)
    
    #iterating in the table
    for index,row in table.iterrows():
        
        #fetching the current code for query
        result = row['Code']
        
        #getting the response from claude api
        response=eval_object.check_response(query,row['Code'])
    
        #Checking the value present in the response generated by the claude
        if 'YES' in response.content:
            response='YES'
        else :
            response='NO'
            
        #Giving the value of response in match column for current row
        table.at[index,'Match']=response
    
    #merging the table dataframe and model_6_query_response in single dataframe
    model_6_layer_query_response= pd.concat([model_6_layer_query_response, table], ignore_index=True)

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_6_layer and are newly initialized: ['encoder.layer.4.output.LayerNorm.bias', 'encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.3.attention.attn.k.bias', 'encoder.layer.2.attention.attn.v.weight', 'encoder.layer.4.attention.attn.k.weight', 'encoder.layer.4.output.dense.bias', 'encoder.layer.4.attention.attn.q.weight', 'encoder.layer.3.attention.attn.v.weight', 'encoder.layer.4.attention.attn.v.bias', 'encoder.layer.3.intermediate.dense.weight', 'encoder.layer.2.attention.attn.k.weight', 'encoder.layer.5.attention.attn.o.weight', 'encoder.layer.4.attention.LayerNorm.bias', 'encoder.layer.5.attention.LayerNorm.bias', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.5.intermediate.dense.bias', 'encoder.layer.5.output.LayerNorm.bias', 'encoder.layer.5.a

(113884, 768)


# 8 Layer Model

In [30]:
# layers to keep in base model
layers=[0,2,3,4,7,8,10,11]

# object of distillation class
prun_object=pruning(layers,teacher_model)

# function of distillation class that makes the model with 8 layer
prun_object.pruning_model()

# Loading the model and generating the embeddings

In [31]:
model=SentenceTransformer('..//models/pruned_model_with_8_layer')
func_object.get_model_embedding(model,"pruned_model_with_8_layer")

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_8_layer and are newly initialized: ['encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.6.attention.attn.q.bias', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.1.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.5.attention.attn.v.bias', 'encoder.layer.5.attention.attn.k.weight', 'encoder.layer.2.attention.attn.k.bias', 'encoder.layer.1.attention.attn.o.weight', 'encoder.layer.1.attention.attn.v.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.2.attention.attn.q.weight', 'encoder.layer.4.attention.LayerNorm.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.3.output.LayerNorm.bias', 'encoder.layer.1.attention.attn.q.weight', 'encoder.layer.2.attention.attn.o.bias', 'encoder.layer.4.attention.attn.v.weight', 'encoder.layer.6.attention.attn.k.weight', 'encoder.layer.5.atte

starting_time : 1713348422.37
ending_time : 1713348865.61
Embedding_shape : (113884, 768)
total_time_taken for embedding generation:  443.23 seconds 


# Size of Model 

In [32]:
size=get_model_size("..//models/pruned_model_with_8_layer")

print(f"Size of 8 layer model is {(size/(1024*1024)):.2f} MB")

Size of 8 layer model is 121.15 MB


# size of embeddings

In [33]:
func_object.size_of_embeddings=get_embedding_size("..//embeddings/embeddings_pruned_model_with_8_layer.npy")

print(f"Size of embeddings of 8 layer model is {(size_of_embeddings/(1024*1024)):.2f} MB")

Size of embeddings of 8 layer model is 333.64 MB


# saving the values in the list

In [154]:
time_embedding_generation.append(443.23)
emebdding_size_MB.append(333.64)
model_size_MB.append(121.15)
layers.append(8)

# Architecture of model with 8 layer

In [43]:
# Print the model summary to view its architecture
print(model_8_layer)


# Number of layers is 8 and Embedding vector dimnesion is 768 

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_8_layer and are newly initialized: ['encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.6.attention.attn.q.bias', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.1.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.5.attention.attn.v.bias', 'encoder.layer.5.attention.attn.k.weight', 'encoder.layer.2.attention.attn.k.bias', 'encoder.layer.1.attention.attn.o.weight', 'encoder.layer.1.attention.attn.v.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.2.attention.attn.q.weight', 'encoder.layer.4.attention.LayerNorm.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.3.output.LayerNorm.bias', 'encoder.layer.1.attention.attn.q.weight', 'encoder.layer.2.attention.attn.o.bias', 'encoder.layer.4.attention.attn.v.weight', 'encoder.layer.6.attention.attn.k.weight', 'encoder.layer.5.atte

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-7): 8 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_fe

# Response on the testing data

In [81]:
# order in which column of databse will be present
column_order = ['Query', 'Docstrings', 'Code','Match']

#creating the dataframe for model_1 which consists of for particular query all top code and
#match column shows whether it is correctly retrieved or not
model_8_layer_query_response=pd.DataFrame(columns=column_order)

model_8_layer=SentenceTransformer('..//models/pruned_model_with_8_layer')

emb_8_layer=np.load("..//embeddings/embeddings_pruned_model_with_8_layer.npy")

print(emb_8_layer.shape)

for i in range(len(queries)):
    #getting the particular query from queries 
    query=queries.loc[i].Questions
    
    # getting the table dataframe that consists of docstring and code 
    #that has top match with query using cosine similarity
    table=eval_object.get_top_code_and_docstring(query,emb_8_layer,model_8_layer,list_data,df)
    
    #adding the column query with all values as 
    table=add_column(table,query)
    
    #adding columns Match that defines whether the code for given query is correctly retrieved or not
    table=add_column(table,'Match')
    
    #reordering the table for better view
    table=reorder_columns(table)
    
    #iterating in the table
    for index,row in table.iterrows():
        
        #fetching the current code for query
        result = row['Code']
        
        #getting the response from claude api
        response=eval_object.check_response(query,row['Code'])
    
        #Checking the value present in the response generated by the claude
        if 'YES' in response.content:
            response='YES'
        else :
            response='NO'
            
        #Giving the value of response in match column for current row
        table.at[index,'Match']=response
    
    #merging the table dataframe and model_1_query_response in single dataframe
    model_8_layer_query_response= pd.concat([model_8_layer_query_response, table], ignore_index=True)

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_8_layer and are newly initialized: ['encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.6.attention.attn.q.bias', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.1.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.5.attention.attn.v.bias', 'encoder.layer.5.attention.attn.k.weight', 'encoder.layer.2.attention.attn.k.bias', 'encoder.layer.1.attention.attn.o.weight', 'encoder.layer.1.attention.attn.v.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.2.attention.attn.q.weight', 'encoder.layer.4.attention.LayerNorm.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.3.output.LayerNorm.bias', 'encoder.layer.1.attention.attn.q.weight', 'encoder.layer.2.attention.attn.o.bias', 'encoder.layer.4.attention.attn.v.weight', 'encoder.layer.6.attention.attn.k.weight', 'encoder.layer.5.atte

(113884, 768)


# 10 layer Model

In [34]:
# layers to keep in base model
layers=[0,2,3,4,5,7,8,9,10,11]

# creating the object of distillation class
prun_object=pruning(layers,teacher_model)

# function of distillation that make the model with 10 layer
prun_object.pruning_model()

# Loading the model and generatting the embeddings

In [35]:
model=SentenceTransformer('..//models/pruned_model_with_10_layer')
func_object.get_model_embedding(model,"pruned_model_with_10_layer")

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_10_layer and are newly initialized: ['encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.6.attention.attn.q.bias', 'encoder.layer.9.output.LayerNorm.bias', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.1.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.9.output.LayerNorm.weight', 'encoder.layer.5.attention.attn.v.bias', 'encoder.layer.5.attention.attn.k.weight', 'encoder.layer.8.attention.attn.o.weight', 'encoder.layer.2.attention.attn.k.bias', 'encoder.layer.1.attention.attn.o.weight', 'encoder.layer.1.attention.attn.v.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.2.attention.attn.q.weight', 'encoder.layer.9.attention.attn.k.weight', 'encoder.layer.8.intermediate.dense.bias', 'encoder.layer.9.attention.attn.k.bias', 'encoder.layer.4.attention.LayerNorm.weight', 'encoder.layer.1.o

starting_time : 1713349285.19
ending_time : 1713349821.06
Embedding_shape : (113884, 768)
total_time_taken for embedding generation:  535.87 seconds 


# size of model

In [36]:
size=func_object.get_model_size("..//models/pruned_model_with_10_layer")

print(f"Size of 10 layer model is {(size/(1024*1024)):.2f} MB")

Size of 10 layer model is 121.15 MB


# size of embeddings

In [37]:
size_of_embeddings=func_object.get_embedding_size("..//embeddings/embeddings_pruned_model_with_10_layer.npy")

print(f"Size of embeddings of 10 layer model is {(size_of_embeddings/(1024*1024)):.2f} MB")

Size of embeddings of 10 layer model is 333.64 MB


# loading the values in different lists

In [155]:
time_embedding_generation.append(535.87)
emebdding_size_MB.append(333.64)
model_size_MB.append(121.15)
layers.append(10)

# Architecture of 10 layer model

In [45]:
# Print the model summary to view its architecture
print(model_10_layer)

# Number of layers is 8 and Embedding vector dimnesion is 768 

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_10_layer and are newly initialized: ['encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.6.attention.attn.q.bias', 'encoder.layer.9.output.LayerNorm.bias', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.1.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.9.output.LayerNorm.weight', 'encoder.layer.5.attention.attn.v.bias', 'encoder.layer.5.attention.attn.k.weight', 'encoder.layer.8.attention.attn.o.weight', 'encoder.layer.2.attention.attn.k.bias', 'encoder.layer.1.attention.attn.o.weight', 'encoder.layer.1.attention.attn.v.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.2.attention.attn.q.weight', 'encoder.layer.9.attention.attn.k.weight', 'encoder.layer.8.intermediate.dense.bias', 'encoder.layer.9.attention.attn.k.bias', 'encoder.layer.4.attention.LayerNorm.weight', 'encoder.layer.1.o

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-9): 10 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_f

# Response on the testing data

In [82]:
# order in which column of databse will be present
column_order = ['Query', 'Docstrings', 'Code','Match']

#creating the dataframe for model_1 which consists of for particular query all top code and
#match column shows whether it is correctly retrieved or not
model_10_layer_query_response=pd.DataFrame(columns=column_order)

model_10_layer=SentenceTransformer('..//models/pruned_model_with_10_layer')

emb_10_layer=np.load("..//embeddings/embeddings_pruned_model_with_10_layer.npy")

print(emb_10_layer.shape)

for i in range(len(queries)):
    #getting the particular query from queries 
    query=queries.loc[i].Questions
    
    # getting the table dataframe that consists of docstring and code 
    #that has top match with query using cosine similarity
    table=eval_object.get_top_code_and_docstring(query,emb_10_layer,model_10_layer,list_data,df)
    
    #adding the column query with all values as 
    table=add_column(table,query)
    
    #adding columns Match that defines whether the code for given query is correctly retrieved or not
    table=add_column(table,'Match')
    
    #reordering the table for better view
    table=reorder_columns(table)
    
    #iterating in the table
    for index,row in table.iterrows():
        
        #fetching the current code for query
        result = row['Code']
        
        #getting the response from claude api
        response=eval_object.check_response(query,row['Code'])
    
        #Checking the value present in the response generated by the claude
        if 'YES' in response.content:
            response='YES'
        else :
            response='NO'
            
        #Giving the value of response in match column for current row
        table.at[index,'Match']=response
    
    #merging the table dataframe and model_1_query_response in single dataframe
    model_10_layer_query_response= pd.concat([model_10_layer_query_response, table], ignore_index=True)

Some weights of MPNetModel were not initialized from the model checkpoint at models/distilled_model_with_10_layer and are newly initialized: ['encoder.layer.5.attention.LayerNorm.weight', 'encoder.layer.5.output.dense.bias', 'encoder.layer.6.attention.attn.q.bias', 'encoder.layer.9.output.LayerNorm.bias', 'encoder.layer.6.intermediate.dense.weight', 'encoder.layer.1.attention.attn.v.bias', 'encoder.layer.4.output.dense.weight', 'encoder.layer.9.output.LayerNorm.weight', 'encoder.layer.5.attention.attn.v.bias', 'encoder.layer.5.attention.attn.k.weight', 'encoder.layer.8.attention.attn.o.weight', 'encoder.layer.2.attention.attn.k.bias', 'encoder.layer.1.attention.attn.o.weight', 'encoder.layer.1.attention.attn.v.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.2.attention.attn.q.weight', 'encoder.layer.9.attention.attn.k.weight', 'encoder.layer.8.intermediate.dense.bias', 'encoder.layer.9.attention.attn.k.bias', 'encoder.layer.4.attention.LayerNorm.weight', 'encoder.layer.1.o

(113884, 768)


# Base Model with 12 Layers

In [46]:
base_model=SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
func_object.get_model_embedding(base_model,"all_mpnet_base_v2")

starting_time : 1713351176.98
ending_time : 1713351830.03
Embedding_shape : (113884, 768)
total_time_taken for embedding generation:  653.05 seconds 


# size of embeddings

In [157]:
size_of_embeddings=func_object.get_embedding_size("..//embeddings/embeddings_all_mpnet_base_v2.npy")

print(f"Size of embeddings of 10 layer model is {(size_of_embeddings/(1024*1024)):.2f} MB")

Size of embeddings of 10 layer model is 333.64 MB


# loading the values in lists

In [158]:
time_embedding_generation.append(653.05)
emebdding_size_MB.append(333.64)
model_size_MB.append(450)
layers.append(12)

# response for the testing data

In [85]:
# order in which column of databse will be present
column_order = ['Query', 'Docstrings', 'Code','Match']

#creating the dataframe for model_1 which consists of for particular query all top code and
#match column shows whether it is correctly retrieved or not
base_model_query_response=pd.DataFrame(columns=column_order)

base_model=SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

emb_base_model=np.load("..//embeddings/embeddings_all_mpnet_base_v2.npy")

print(emb_10_layer.shape)

for i in range(len(queries)):
    #getting the particular query from queries 
    query=queries.loc[i].Questions
    
    # getting the table dataframe that consists of docstring and code 
    #that has top match with query using cosine similarity
    table=eval_object.get_top_code_and_docstring(query,emb_base_model,base_model,list_data,df)
    
    #adding the column query with all values as 
    table=add_column(table,query)
    
    #adding columns Match that defines whether the code for given query is correctly retrieved or not
    table=add_column(table,'Match')
    
    #reordering the table for better view
    table=reorder_columns(table)
    
    #iterating in the table
    for index,row in table.iterrows():
        
        #fetching the current code for query
        result = row['Code']
        
        #getting the response from claude api
        response=eval_object.check_response(query,row['Code'])
    
        #Checking the value present in the response generated by the claude
        if 'YES' in response.content:
            response='YES'
        else :
            response='NO'
            
        #Giving the value of response in match column for current row
        table.at[index,'Match']=response
    
    #merging the table dataframe and model_1_query_response in single dataframe
    base_model_query_response= pd.concat([base_model_query_response, table], ignore_index=True)

(113884, 768)


# MAP of 4 layer model

In [197]:
total_yes_model_4_layer = (model_4_layer_query_response['Match'] == 'YES').sum()
total_no_model_4_layer = (model_4_layer_query_response['Match'] == 'NO').sum()

accuracy_4_layer=(total_yes_model_4_layer/(total_yes_model_4_layer+total_no_model_4_layer))*100

print(f"MAP@10 (mean average precision) of 4 layer model : {accuracy_4_layer}")

accuracy.append(int(accuracy_4_layer))

MAP@10 (mean average precision) of 4 layer model : 61.40350877192983


# MAP of 6 layer model

In [198]:
total_yes_model_6_layer = (model_6_layer_query_response['Match'] == 'YES').sum()
total_no_model_6_layer = (model_6_layer_query_response['Match'] == 'NO').sum()

accuracy_6_layer=(total_yes_model_6_layer/(total_yes_model_6_layer+total_no_model_6_layer))*100

print(f"MAP@10 (mean average precision) of 6 layer model : {accuracy_6_layer}")

accuracy.append(int(accuracy_6_layer))

MAP@10 (mean average precision) of 6 layer model : 67.71929824561404


# MAP for 8 layer model

In [199]:
total_yes_model_8_layer = (model_8_layer_query_response['Match'] == 'YES').sum()
total_no_model_8_layer = (model_8_layer_query_response['Match'] == 'NO').sum()

accuracy_8_layer=(total_yes_model_8_layer/(total_yes_model_8_layer+total_no_model_8_layer))*100

print(f"MAP@10 (mean average precision) of 8 layer model : {accuracy_8_layer}")

accuracy.append(int(accuracy_8_layer))

MAP@10 (mean average precision) of 8 layer model : 64.3859649122807


# MAP for 10 Layer model

In [200]:
total_yes_model_10_layer = (model_10_layer_query_response['Match'] == 'YES').sum()
total_no_model_10_layer = (model_10_layer_query_response['Match'] == 'NO').sum()

accuracy_10_layer=(total_yes_model_10_layer/(total_yes_model_10_layer+total_no_model_10_layer))*100

print(f"MAP@10 (mean average precision) of 10 layer model : {accuracy_10_layer}")

accuracy.append(int(accuracy_10_layer))

MAP@10 (mean average precision) of 10 layer model : 64.73684210526316


#  MAP of original model

In [201]:
total_yes_model_12_layer = (base_model_query_response['Match'] == 'YES').sum()
total_no_model_12_layer = (base_model_query_response['Match'] == 'NO').sum()

accuracy_12_layer=(total_yes_model_12_layer/(total_yes_model_12_layer+total_no_model_12_layer))*100

print(f"MAP@10 (mean average precision) of 12 layer model : {accuracy_12_layer}")

accuracy.append(int(accuracy_12_layer))

MAP@10 (mean average precision) of 12 layer model : 66.14035087719297


In [210]:
print(model_size_MB)

[202.27, 148.19, 121.15, 121.15, 450]


# Graph between Layers and Accuracy

In [206]:
fig_model_size = go.Figure()
fig_model_size.add_trace(go.Scatter(x=layers, y=accuracy, mode='lines+markers'))
fig_model_size.update_layout(title='Layers vs. Accuracy', xaxis_title='Layers of Model', yaxis_title='Accuracy')
fig_model_size.show()

# Graph between layers and model size

In [205]:
fig_model_size = go.Figure()
fig_model_size.add_trace(go.Scatter(x=layers, y=model_size_MB, mode='lines+markers'))
fig_model_size.update_layout(title='Layers vs. Model size ', xaxis_title='Layers of Model', yaxis_title='Model size in MB')
fig_model_size.show()

# Graph between the layers and time taken to generate the embeddings

In [211]:
fig_model_size = go.Figure()
fig_model_size.add_trace(go.Scatter(x=layers, y=time_embedding_generation, mode='lines+markers'))
fig_model_size.update_layout(title='Layers vs time taken to generate to Embeddings ', xaxis_title='Layers of Model', yaxis_title='time taken to generate embeddings (sec)')
fig_model_size.show()